In [1]:
# import some libraries
import json
import uuid 
from elasticsearch import helpers
from elasticsearch import Elasticsearch

## read the josn file

In [2]:
# read the places data to insert them to elasticsearch
with open("/Users/emadarmiti/Desktop/cap-s5/places_ranking/osm_data/test.json", 'r') as data_file:
    data = json.load(data_file)

### slice the data

In [6]:
# general look at the first 3 places
data['features'][:3]

[{'id': '3',
  'type': 'Feature',
  'properties': {'name': 'Kibler Park'},
  'geometry': {'type': 'Point', 'coordinates': [-78.6723302, 43.1655945]}},
 {'id': '4',
  'type': 'Feature',
  'properties': {'name': 'Inn By The Park'},
  'geometry': {'type': 'Point', 'coordinates': [-75.1791306, 44.8968001]}},
 {'id': '5',
  'type': 'Feature',
  'properties': {'name': 'Northwood Elementary School'},
  'geometry': {'type': 'Point', 'coordinates': [-78.7428759, 42.863407]}}]

In [4]:
# slice the places data, the original dict has many keys, we just take the places data list
data = data['features']

In [6]:
# define the data which is ready to insert to elastic search
data_elasticsearch = []

# go over each place slice the name and the location
for place in data:
    
    # dfine the place dict
    place_info = {}
    
    # add the place name to the dict
    place_info['name'] = place['properties']['name']
    
    # add the location to the dict, lat and lon
    place_info["location"]={'lat':place['geometry']['coordinates'][1],
                             'lon':place['geometry']['coordinates'][0]}
    
    # append the current place info to the list
    data_elasticsearch.append(place_info)

### conect to the elasticsearch on the cloud

In [7]:
# connect to the elasticsearch endpoint.  
elastc_seach = Elasticsearch("elasticsearch end point"
                            ,http_auth=('name', 'pass'))

# define the index name
index_name = "places"

### create the indeax

In [8]:
# define the mapping (settings of the index)
settings = {
  "mappings": {
        "dynamic": "strict", # cannot add extra fields 
        "properties": { # the properites of the fields
          "name": {"type": "text"}, # the type of the field name
          "location": {"type": "geo_point"}} # the type of the field location
        }
      }

# create the index  
elastc_seach.indices.create(index = index_name, ignore = 400, body = settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'places'}

### add meta data to data and bulk insertion to elasticsearch


In [9]:
# this list contains the places data including the bulk insertion meta data
actions = []

# go over each place and add the meta data
for place in data_elasticsearch:    
    
    # create a new dict that contains the place data and the meta data
    place_meta = { '_op_type': 'index',
                   '_index': index_name, # add the index name
                   '_type': '_doc', # the type of passed data
                   '_id': uuid.uuid1(), # add rendom id
                   '_source': place} # pass the data

    # append the place with meta dict to the actions list
    actions.append(place_meta)

# bulk insertion for the actions list
helpers.bulk(es, actions)

(44467, [])